In [2]:
import collections
import copy
import os

import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from absl import app, flags
from skimage import io
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from tqdm import tqdm

In [21]:
root_dir = os.path.join('pacs_dataset', 'train')
os.path.isdir(root_dir)
dataset = []
domain_set = set()
category_set = set()
cnt = 0

TypeError: isdir() got an unexpected keyword argument 'maxsplit'

In [29]:
for root, dirs, files in os.walk(root_dir, topdown=True):
    if files and files[0] != '.DS_Store':
        _, domain, category = root.rsplit('/', maxsplit=2)
        print(domain)
        print(category)
        domain_set.add(domain)
        category_set.add(category)
        pbar = tqdm(files)
        for name in pbar:
            pbar.set_description('Processing Folder: domain=%s, category=%s' %
                               (domain, category))
            img_array = io.imread(os.path.join(root, name))
            dataset.append((img_array, domain, category))

Processing Folder: domain=cartoon, category=dog:   3%|▎         | 12/343 [00:00<00:02, 116.11it/s]

cartoon
dog


Processing Folder: domain=cartoon, category=dog: 100%|██████████| 343/343 [00:01<00:00, 219.00it/s]
Processing Folder: domain=cartoon, category=person:   4%|▍         | 15/364 [00:00<00:02, 144.67it/s]

cartoon
person


Processing Folder: domain=cartoon, category=person: 100%|██████████| 364/364 [00:01<00:00, 209.91it/s]
Processing Folder: domain=cartoon, category=giraffe:   7%|▋         | 22/314 [00:00<00:01, 212.75it/s]

...

Processing Folder: domain=sketch, category=house: 100%|██████████| 75/75 [00:00<00:00, 146.85it/s]
Processing Folder: domain=sketch, category=horse:   3%|▎         | 19/736 [00:00<00:03, 184.12it/s]

sketch
horse


Processing Folder: domain=sketch, category=horse: 100%|██████████| 736/736 [00:04<00:00, 176.31it/s]
Processing Folder: domain=sketch, category=elephant:   3%|▎         | 17/674 [00:00<00:03, 167.43it/s]

sketch
elephant


Processing Folder: domain=sketch, category=elephant: 100%|██████████| 674/674 [00:03<00:00, 180.62it/s]


In [38]:
dataset[0][2]

'dog'